In [ ]:
#Importo librerias necesiaras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas_profiling as pdf
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows',30)

In [ ]:
#Lectura de datos para train y testing
df_train = pd.read_csv('../data_input/Entrenamieto_ECI_2020.csv')
df_test = pd.read_csv('../data_input/Validacion_ECI_2020.csv')

In [ ]:
#Reclasificacion de los valores
df_train['Stage'] = df_train['Stage'].map({'Closed Lost': 0 , 'Closed Won' : 1,'Proposal': 0,'Negotiation': 0,'Qualification': 0})
df_test['Stage'] = -1

In [ ]:
#Unifico tanto train y testing para unificar crierios y optimizar codigo
df = df_train.append(df_test,sort=False,ignore_index=False)

In [ ]:
#Elimino columnas insignificantes
df.drop(['ID','Actual_Delivery_Date','ASP_(converted)_Currency','Last_Activity','Prod_Category_A','Month','Total_Amount_Currency','Delivery_Year','Opportunity_Name'],axis=1,inplace=True)

In [ ]:
#Convierto variables fechas a tipo de datos datetime de pandas
df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'])
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'])
df['Quote_Expiry_Date'] = pd.to_datetime(df['Quote_Expiry_Date'])
df['Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'])
df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'])
df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'])

In [ ]:
#Me creo una variable que cuente la cantidad de productos por oportunidad
df_o = df.groupby('Opportunity_ID')['Opportunity_ID'].count()
df_o.name = 'OpporCount'

In [ ]:
#Preparo el conjunto de datos que es estatico de la oportunidad
df_t_O = df.drop([
'Product_Family',
'Product_Name',
'Planned_Delivery_Start_Date',
'Planned_Delivery_End_Date',
'Delivery_Quarter',
'TRF',
'Total_Amount',
'ASP',
'ASP_(converted)'],axis=1).drop_duplicates(keep='first')

In [ ]:
#Abrir las variables fechas por mes,dia,dayoffwee,qurter,weekofyear
df_t_O['Account_Created_Date_month'] = pd.DatetimeIndex(df_t_O['Account_Created_Date']).month
df_t_O['Account_Created_Date_day'] = pd.DatetimeIndex(df_t_O['Account_Created_Date']).day
df_t_O['Account_Created_Date_dayofweek'] = df_t_O['Account_Created_Date'].dt.dayofweek
df_t_O['Account_Created_Date_quarter'] = df_t_O['Account_Created_Date'].dt.quarter
df_t_O['Account_Created_Date_weekofYear'] = df_t_O['Account_Created_Date'].dt.weekofyear
df_t_O['Account_Created_Date_month_bimester'] = df_t_O['Account_Created_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})


df_t_O['Last_Modified_Date_month'] = pd.DatetimeIndex(df_t_O['Last_Modified_Date']).month
df_t_O['Last_Modified_Date_day'] = pd.DatetimeIndex(df_t_O['Last_Modified_Date']).day
df_t_O['Last_Modified_Date_dayofweek'] = df_t_O['Last_Modified_Date'].dt.dayofweek
df_t_O['Last_Modified_Date_quater'] = df_t_O['Last_Modified_Date'].dt.quarter
df_t_O['Last_Modified_Date_weekofYear'] = df_t_O['Last_Modified_Date'].dt.weekofyear
df_t_O['Last_Modified_Date_month_bimester'] = df_t_O['Last_Modified_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})

df_t_O['Opportunity_Created_Date_month'] = pd.DatetimeIndex(df_t_O['Opportunity_Created_Date']).month
df_t_O['Opportunity_Created_Date_day'] = pd.DatetimeIndex(df_t_O['Opportunity_Created_Date']).day
df_t_O['Opportunity_Created_Date_dayofweek'] = df_t_O['Opportunity_Created_Date'].dt.dayofweek
df_t_O['Opportunity_Created_Date_quater'] = df_t_O['Opportunity_Created_Date'].dt.quarter
df_t_O['Opportunity_Created_Date_weekOfyear'] = df_t_O['Opportunity_Created_Date'].dt.weekofyear
df_t_O['Opportunity_Created_Date_weekday'] = df_t_O['Opportunity_Created_Date'].dt.weekday
df_t_O['Opportunity_Created_Date_bimester'] = df_t_O['Opportunity_Created_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})

df_t_O['Quote_Expiry_Date_month'] = pd.DatetimeIndex(df_t_O['Quote_Expiry_Date']).month
df_t_O['Quote_Expiry_Date_day'] = pd.DatetimeIndex(df_t_O['Quote_Expiry_Date']).day
df_t_O['Quote_Expiry_Date_dayofweek'] = df_t_O['Quote_Expiry_Date'].dt.dayofweek
df_t_O['Quote_Expiry_Date_quater'] = df_t_O['Quote_Expiry_Date'].dt.quarter
df_t_O['Quote_Expiry_Date_weekofYear'] = df_t_O['Quote_Expiry_Date'].dt.weekofyear
df_t_O['Quote_Expiry_Date_weekday'] = df_t_O['Quote_Expiry_Date'].dt.weekday
df_t_O['Quote_Expiry_Date_bimester'] = df_t_O['Quote_Expiry_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})


In [ ]:
#Creo variables que representen diferencias de dias
df_t_O['lastModified_AccountCreated_dif'] =  (df_t_O['Last_Modified_Date'] - df_t_O['Account_Created_Date']).dt.days 
df_t_O['opportuniy_AccountCreated_dif'] =  (df_t_O['Opportunity_Created_Date'] - df_t_O['Account_Created_Date']).dt.days
df_t_O['lastModified_Oppor_dif'] =  (df_t_O['Last_Modified_Date'] - df_t_O['Opportunity_Created_Date']).dt.days 
df_t_O['Quote_Oppor_dif'] =  (df_t_O['Quote_Expiry_Date'] - df_t_O['Opportunity_Created_Date']).dt.days
df_t_O['lastModified_Account_dif'] =  (df_t_O['Last_Modified_Date'] - df_t_O['Account_Created_Date']).dt.days
df_t_O['Quote_lastModified_dif'] =  (df_t_O['Quote_Expiry_Date'] - df_t_O['Last_Modified_Date']).dt.days
df_t_O.drop(list(df_t_O.select_dtypes(include=['datetime64[ns]']).columns.values),axis=1 , inplace=True)

In [ ]:
#Reemplazo los valores no enteros por un default
df_t_O['Sales_Contract_No'] = df_t_O['Sales_Contract_No'].replace(['None'], -1).astype(str).astype(int)
df_t_O['Price'] = df_t_O['Price'].replace(['None'], -1)
df_t_O['Price'] = df_t_O['Price'].replace(['Other'], -10).astype(str).astype(float)

In [ ]:
#Reemplazo los valores no enteros por la media
df_t_O['Sales_Contract_No_1'] = df_t_O['Sales_Contract_No'].replace(-1,df_t_O[df_t_O['Sales_Contract_No'] != -1]['Sales_Contract_No'].mean())

In [ ]:
#Reemplazo los valores no enteros por la media
df_t_O['Price_1'] = df_t_O['Price'].replace([-1,-10],df_t_O[~df_t_O['Price'].isin([-1,-10])]['Price'].mean())

In [ ]:
#Reemplazo los valore no enteros por la media
df_t_O['Price'] = df_t_O['Price'].replace(['None','Other'], -1 ).astype(str).astype(float)
df_t_O['Price'] = df_t_O['Price'].replace(-1,df_t_O[df_t_O['Price'] != -1]['Price'].mean()).astype(str).astype(float)

In [ ]:
#Un intento de crear mas variables por cada oportunidad, no mejora la performance

# brand = df_t_O.groupby('Brand')['Opportunity_ID'].count()
# brand.name = 'brand_name_j'

# region = df_t_O.groupby('Region')['Opportunity_ID'].count()
# region.name = 'region_name_j'

# territorio = df_t_O.groupby('Territory')['Opportunity_ID'].count()
# territorio.name = 'territorio_name_j'

# account = df_t_O.groupby('Account_Name')['Opportunity_ID'].count()
# account.name = 'account_name_j'

# op_ow = df_t_O.groupby('Opportunity_Owner')['Opportunity_ID'].count()
# op_ow.name = 'oppor_owner_j'

# ac_ow = df_t_O.groupby('Account_Owner')['Opportunity_ID'].count()
# ac_ow.name = 'account_owner_j'

# df_t_O = pd.merge(df_t_O,
#                  account,
#                  on='Account_Name')

# df_t_O = pd.merge(df_t_O,
#                  op_ow,
#                  on='Opportunity_Owner')

# df_t_O = pd.merge(df_t_O,
#                  brand,
#                  on='Brand')

# df_t_O = pd.merge(df_t_O,
#                  region,
#                  on='Region')

# df_t_O = pd.merge(df_t_O,
#                  territorio,
#                  on='Territory')

# df_t_O = pd.merge(df_t_O,
#                  ac_ow,
#                  on='Account_Owner')

# monto = df_t_O.groupby('Account_Name')['Total_Taxable_Amount','Price'].agg([np.min, np.max, np.mean,np.median,np.sum])

# monto.columns = ['_'.join(col) for col in monto.columns]

# df_t_O = pd.merge(df_t_O,
#                  monto,
#                  on='Account_Name')

In [ ]:
#Creo valores dummys
for col in list(df_t_O.select_dtypes(include=['object']).columns.values):
           df_t_O  = pd.concat([df_t_O,pd.get_dummies(df_t_O[col],prefix=[col],drop_first=True,dummy_na=True)], axis=1) 
df_t_O.drop(list(df_t_O.select_dtypes(include=['object']).columns.values),axis = 1,inplace = True)

In [ ]:
#Preparo el conjunto de variables que describen a los productos de la oportunidad
df_t_g = df[[
'Opportunity_ID',
'Product_Family',
'Product_Name',
'Planned_Delivery_Start_Date',
'Planned_Delivery_End_Date',
'Account_Created_Date',
'Delivery_Quarter',
'TRF',
'Total_Amount',
'ASP',
'Price',
'Total_Taxable_Amount',
'ASP_(converted)']]

In [ ]:
#Creacion de nuevas variables de fechas

df_t_g['Planned_Delivery_Start_Date_month'] = pd.DatetimeIndex(df_t_g['Planned_Delivery_Start_Date']).month
df_t_g['Planned_Delivery_End_Date_month'] = pd.DatetimeIndex(df_t_g['Planned_Delivery_End_Date']).month

df_t_g['Planned_Delivery_Start_Date_day'] = pd.DatetimeIndex(df_t_g['Planned_Delivery_Start_Date']).day
df_t_g['Planned_Delivery_End_Date_day'] = pd.DatetimeIndex(df_t_g['Planned_Delivery_End_Date']).day

df_t_g['Planned_Delivery_Start_Date_dayofweek'] = df_t_g['Planned_Delivery_Start_Date'].dt.dayofweek
df_t_g['Planned_Delivery_End_Date_dayofweek'] = df_t_g['Planned_Delivery_End_Date'].dt.dayofweek

df_t_g['Planned_Delivery_Start_Date_quater'] = df_t_g['Planned_Delivery_Start_Date'].dt.quarter
df_t_g['Planned_Delivery_End_Date_quater'] = df_t_g['Planned_Delivery_End_Date'].dt.quarter

df_t_g['Planned_Delivery_Start_Date_weekday'] = df_t_g['Planned_Delivery_Start_Date'].dt.weekday
df_t_g['Planned_Delivery_End_Date_weekday'] = df_t_g['Planned_Delivery_End_Date'].dt.weekday

df_t_g['Planned_Delivery_Start_Date_weekofYear'] = df_t_g['Planned_Delivery_Start_Date'].dt.weekofyear
df_t_g['Planned_Delivery_End_Date_weekofYear'] = df_t_g['Planned_Delivery_End_Date'].dt.weekofyear

df_t_g['Planned_Delivery_Start_Date_bimester'] = df_t_g['Planned_Delivery_Start_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})
df_t_g['Planned_Delivery_End_Date_bimester'] = df_t_g['Planned_Delivery_End_Date_month'].map({1:1,2:1 , 3:2,4:2 , 5:3,6:3 , 7:4,8:4 , 9:5,10:5 , 11:6,12:6})


df_t_g['Planned_Delivery_diferencia'] =  (df_t_g['Planned_Delivery_End_Date'] - df_t_g['Planned_Delivery_Start_Date']).dt.days 
df_t_g['end_account_diferencia'] =  (df_t_g['Planned_Delivery_End_Date'] - df_t_g['Account_Created_Date']).dt.days
df_t_g['start_account_diferencia'] =  (df_t_g['Planned_Delivery_Start_Date'] - df_t_g['Account_Created_Date']).dt.days


df_t_g.drop(['Planned_Delivery_Start_Date','Planned_Delivery_End_Date','Account_Created_Date'],axis=1 , inplace=True)

In [ ]:
#Agrupo las variables numericas y creo algunas medidas

df_t_g_f = df_t_g[['Planned_Delivery_Start_Date_month',
'Planned_Delivery_End_Date_month',
'Planned_Delivery_Start_Date_day',
'Planned_Delivery_End_Date_day',
'Planned_Delivery_Start_Date_dayofweek',
'Planned_Delivery_End_Date_dayofweek',
'Planned_Delivery_Start_Date_quater',
'Planned_Delivery_End_Date_quater',
'Planned_Delivery_End_Date_weekday',
'Planned_Delivery_Start_Date_weekday',
'Planned_Delivery_Start_Date_weekofYear',
'Planned_Delivery_End_Date_weekofYear',
'Planned_Delivery_End_Date_bimester',
'Planned_Delivery_Start_Date_bimester',
'Planned_Delivery_diferencia',
'end_account_diferencia',
'start_account_diferencia',
'Opportunity_ID']
].groupby('Opportunity_ID').agg([np.min, np.max, np.mean,np.median,np.sum,])
df_t_g_f.columns = ['_'.join(col) for col in df_t_g_f.columns]

In [ ]:
#Agrupo las variables numericas y creo algunas medidas

#fechas = list(df_train.select_dtypes(include=['datetime64[ns]']).columns.values)
df_t_g = df_t_g[['TRF','Total_Amount','ASP','ASP_(converted)','Price','Total_Taxable_Amount','Opportunity_ID']].groupby('Opportunity_ID').agg([np.min, np.max, np.mean,np.median,np.sum])
df_t_g.columns = ['_'.join(col) for col in df_t_g.columns]

In [ ]:
#Creo dummies a partir de la variable product family
pf = pd.concat([df['Opportunity_ID'],pd.get_dummies(df['Product_Family'],prefix=['PF_A'],drop_first=True,dummy_na=True)], axis=1)
pf = pf.groupby('Opportunity_ID').agg(np.sum)

In [ ]:
#Creo dummies a partir de la variable product name
pn = pd.concat([df['Opportunity_ID'],pd.get_dummies(df['Product_Name'],prefix=['PN_A'],drop_first=True,dummy_na=True)], axis=1)
pn = pn.groupby('Opportunity_ID').agg(np.sum)

In [ ]:
#Creo dummies a partir de la variable Deivery Quarter
dq = pd.concat([df['Opportunity_ID'],pd.get_dummies(df['Delivery_Quarter'],prefix=['DQ_A'],drop_first=True,dummy_na=True)], axis=1)
dq = dq.groupby('Opportunity_ID').agg(np.sum)

In [ ]:
#Concateno todos los sets y unifico
df_t_G = pd.concat([df_t_g,pf,pn,dq,df_t_g_f],axis=1)

In [ ]:
df_t_G.reset_index(drop=True, inplace=True)
df_t_O.reset_index(drop=True, inplace=True)
df_o.reset_index(drop=True, inplace=True)

In [ ]:
#Estructura final de datos
df_final = pd.concat([df_t_G, df_t_O,df_o], axis=1)

In [ ]:
#Elimino variables independientes correlacionadas
c = df_final.corr().abs()
#s = c.unstack()

upper = c.where(np.triu(np.ones(c.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df_final.drop(df_final[to_drop], axis=1,inplace=True)

In [ ]:
df_final.to_csv('../data_processed/df_final.csv',index=False)

In [ ]:
df_final_train = df_final[df_final['Stage'] != -1 ]

In [ ]:
df_final_test = df_final[df_final['Stage'] == -1 ]
df_final_test.drop('Stage',axis=1,inplace=True)

In [ ]:
df_final_train.shape

In [ ]:
df_final_test.shape

# LIGHTGBM

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import itertools
from lightgbm import LGBMClassifier

In [ ]:
y = df_final_train['Stage']
X = df_final_train.drop('Stage',axis=1)
#X = df_final_train.drop(['Stage','Opportunity_ID'],axis=1)
#ido = df_final_train['Opportunity_ID']
X.columns = np.arange(X.shape[1])

In [ ]:
def expand_grid(data_dict):
  rows = itertools.product(*data_dict.values())
  return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [ ]:
cv_data = lgb.Dataset(X, label = y)

In [ ]:
#lightgbm parameters
parameters = {
    'max_depth': np.arange(13,14,1),
    'feature_fraction': np.arange(0.2,0.22,0.005),
    'bagging_fraction': np.arange(0.9,0.95,0.005),
    'lambda': np.arange(0.2,0.3,0.005),
    'application': ['binary'],
    'boosting': ['gbdt'],
    #'num_boost_round' : np.arange(100,1000,100),
    'learning_rate': np.arange(0.1,0.2,0.005),
    'objective': ['binary'],
    'metric': ['binary_logloss'],
    'is_unbalance': ['false'],
    'num_leaves': np.arange(18,20,1),
    'bagging_freq': np.arange(16,18,1),
    'num_boost_round': np.arange(1900,2020,10)
    #'verbose': [0]    
}

parametersdf = expand_grid(parameters).sample(frac=1)

In [ ]:
#parametersdf.drop('num_boost_round',axis=1).iloc[i].to_dict()

In [ ]:
auc = []
for i in range(len(parametersdf)):  
  modelo = lgb.cv(parametersdf.drop('num_boost_round',axis=1).iloc[i].to_dict(), cv_data, nfold=5,early_stopping_rounds = 20,num_boost_round=parametersdf.iloc[i].to_dict()['num_boost_round'])
  print(min(modelo['binary_logloss-mean']))
  #print(int(np.argmin(modelo['binary_logloss-mean']) + 1))
  #print(i)
  auc.append([min(modelo['binary_logloss-mean']),int(np.argmin(modelo['binary_logloss-mean']) + 1),i]) 

In [ ]:
resultado = pd.DataFrame(auc,columns=['auc','it','loc'])
resultado[resultado['auc'] == resultado['auc'].min()]

In [ ]:
parametersdf.iloc[resultado[resultado['auc'] == resultado['auc'].min()]['loc']]

In [ ]:
params = parametersdf.iloc[resultado[resultado['auc'] == resultado['auc'].min()]['loc']]
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params['max_depth']).item(),
    'lambda' : np.array(params['lambda']).item(),
    'num_leaves': np.array(params['num_leaves']).item(),
    'feature_fraction': np.array(params['feature_fraction']).item(),
    'bagging_fraction': np.array(params['bagging_fraction']).item(),
    'bagging_freq': np.array(params['bagging_freq']).item(),
    'learning_rate': np.array(params['learning_rate']).item(),
    'verbose': 0
}

In [ ]:
model = lgb.train(parameters,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=resultado[resultado['auc'] == resultado['auc'].min()]['it'].item(),
                       #early_stopping_rounds=100
                      )

In [ ]:
pred = model.predict(df_final_test)

In [ ]:
output = pd.DataFrame({'id': df_final_test['Opportunity_ID'], 'target': pred})
output.sort_values(by=['id'],inplace=True)
output.to_csv('../results/'+str(resultado[resultado['auc'] == resultado['auc'].min()]['auc'].item()) + '.csv', index=False,header=False)

In [ ]:
from mlflow import log_metric, log_param, log_artifact , create_experiment ,get_experiment_by_name ,start_run ,end_run ,set_experiment
import mlflow.sklearn 

In [ ]:
parametrs = {'application': 'binary',
 'bagging_fraction': 0.9500000000000002,
 'bagging_freq': 16,
 'boosting': 'gbdt',
 'feature_fraction': 0.2,
 'lambda': 0.1,
 'learning_rate': 0.1,
 'max_depth': 12,
 'metric': 'binary_logloss',
 'num_leaves': 19,
 'objective': 'binary',
 'verbose': 0}

In [ ]:
expermientName = 'Competencia_AlixPartner_Oportunidad_compras'

try:
    experiment_id = create_experiment(name=expermientName)
except:
    experiment_id = get_experiment_by_name(name=expermientName).experiment_id

set_experiment(expermientName)
with start_run():
    log_param("parameters",parameters)
    log_metric("logloss",resultado[resultado['auc'] == resultado['auc'].min()]['auc'].item())
    log_artifact('../data_processed/df_final.csv')
    #log_artifact('../results/'+str(resultado[resultado['auc'] == resultado['auc'].min()]['auc'].item()) + '.csv')
    log_artifact('../results/0.07738.csv')
    log_artifact('Script_1.ipynb')
    #mlflow.sklearn.save_model(model,)

# TOP5

In [ ]:
top5 = resultado.sort_values('auc').head(5)

In [ ]:
top5

In [ ]:
params1 = parametersdf.iloc[int(top5.iloc[0]['loc'])]
parameters1 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params1['max_depth']).item(),
    'lambda' : np.array(params1['lambda']).item(),
    'num_leaves': np.array(params1['num_leaves']).item(),
    'feature_fraction': np.array(params1['feature_fraction']).item(),
    'bagging_fraction': np.array(params1['bagging_fraction']).item(),
    'bagging_freq': np.array(params1['bagging_freq']).item(),
    'learning_rate': np.array(params1['learning_rate']).item(),
    'verbose': 0
}

params2 = parametersdf.iloc[int(top5.iloc[1]['loc'])]
parameters2 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params2['max_depth']).item(),
    'lambda' : np.array(params2['lambda']).item(),
    'num_leaves': np.array(params2['num_leaves']).item(),
    'feature_fraction': np.array(params2['feature_fraction']).item(),
    'bagging_fraction': np.array(params2['bagging_fraction']).item(),
    'bagging_freq': np.array(params2['bagging_freq']).item(),
    'learning_rate': np.array(params2['learning_rate']).item(),
    'verbose': 0
}

params3 = parametersdf.iloc[int(top5.iloc[2]['loc'])]
parameters3 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params3['max_depth']).item(),
    'lambda' : np.array(params3['lambda']).item(),
    'num_leaves': np.array(params3['num_leaves']).item(),
    'feature_fraction': np.array(params3['feature_fraction']).item(),
    'bagging_fraction': np.array(params3['bagging_fraction']).item(),
    'bagging_freq': np.array(params3['bagging_freq']).item(),
    'learning_rate': np.array(params3['learning_rate']).item(),
    'verbose': 0
}

params4 = parametersdf.iloc[int(top5.iloc[3]['loc'])]
parameters4 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params4['max_depth']).item(),
    'lambda' : np.array(params4['lambda']).item(),
    'num_leaves': np.array(params4['num_leaves']).item(),
    'feature_fraction': np.array(params4['feature_fraction']).item(),
    'bagging_fraction': np.array(params4['bagging_fraction']).item(),
    'bagging_freq': np.array(params4['bagging_freq']).item(),
    'learning_rate': np.array(params4['learning_rate']).item(),
    'verbose': 0
}

params5 = parametersdf.iloc[int(top5.iloc[4]['loc'])]
parameters5 = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'gbdt',
    'max_depth' : np.array(params5['max_depth']).item(),
    'lambda' : np.array(params5['lambda']).item(),
    'num_leaves': np.array(params5['num_leaves']).item(),
    'feature_fraction': np.array(params5['feature_fraction']).item(),
    'bagging_fraction': np.array(params5['bagging_fraction']).item(),
    'bagging_freq': np.array(params5['bagging_freq']).item(),
    'learning_rate': np.array(params5['learning_rate']).item(),
    'verbose': 0
}

In [ ]:
model1 = lgb.train(parameters1,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=int(resultado.sort_values('auc').iloc[0]['it'].item())
                       #early_stopping_rounds=100
                      )

model2 = lgb.train(parameters2,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=int(resultado.sort_values('auc').iloc[1]['it'].item())
                       #early_stopping_rounds=100
                      )

model3 = lgb.train(parameters3,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=int(resultado.sort_values('auc').iloc[2]['it'].item())
                       #early_stopping_rounds=100
                      )

model4 = lgb.train(parameters4,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=int(resultado.sort_values('auc').iloc[3]['it'].item())
                       #early_stopping_rounds=100
                      )

model5 = lgb.train(parameters5,
                       cv_data,
                       #valid_sets=lgb_eval,
                       num_boost_round=int(resultado.sort_values('auc').iloc[4]['it'].item())
                       #early_stopping_rounds=100
                      )

In [ ]:
pred1 = model1.predict(df_final_test)
pred2 = model2.predict(df_final_test)
pred3 = model3.predict(df_final_test)
pred4 = model4.predict(df_final_test)
pred5 = model5.predict(df_final_test)

output = pd.DataFrame({'id': df_final_test['Opportunity_ID'], 'target': (pred1+pred2+pred3+pred4+pred5)/5})
output.sort_values(by=['id'],inplace=True)
output.to_csv('vote5.csv', index=False,header=False)